# TP3 - Grupo 20
###  Dezembro, 4, 2024

Afonso Martins Campos Fernandes - A102940

Luís Filipe Pinheiro Silva - A105530

## Cifra A5/1 - SFOTS

Considere de novo o 1º problema do trabalho TP2  relativo à descrição da cifra $$\,\mathsf{A5/1}$$ e o FOTS usando BitVec’s que aì foi definido para a componente do gerador de chaves. Ignore a componente de geração final da chave e restrinja o modelo aos três LFSR’s. 
Sejam $$\,\mathsf{X}_0, \mathsf{X}_1, \mathsf{X}_2\,$$ as variáveis que determinam os estados dos três LFSR’s que ocorrem neste modelo. Como condição inicial  e condição de erro use os predicados

$$\,\mathsf{I} \;\equiv\; (\mathsf{X}_0 > 0)\,\land\,(\mathsf{X}_1 > 0)\,\land\,(\mathsf{X}_2 > 0)\quad e \quad \mathsf{E}\;\equiv\;\neg\,\mathsf{I}$$

In [277]:
from pysmt.shortcuts import *
from z3 import *
from random import getrandbits

a. Codifique em “z3”  o SFOTS assim definido.

In [278]:
def states(i):
    state = {}

    state['LFSR1'] = BitVec('LFSR1' + str(i),19)
    state['LFSR2'] = BitVec('LFSR2' + str(i),22)
    state['LFSR3'] = BitVec('LFSR3' + str(i),23)

    return state

In [279]:
def init(state):
    return And(state['LFSR1'] > 0,state['LFSR2'] > 0,state['LFSR3'] > 0)

In [280]:
def error(state):
    return Not(And(state['LFSR1'] > 0,state['LFSR2'] > 0,state['LFSR3'] > 0))

In [281]:
def transition(atual,prox):
    cBit1 = Extract(8, 8, atual['LFSR1'])
    cBit2 = Extract(10, 10, atual['LFSR2'])
    cBit3 = Extract(10, 10, atual['LFSR3'])

    t1y = And(prox['LFSR1'] == Concat(Extract(18, 18, atual['LFSR1']) ^ 
                                      Extract(17, 17, atual['LFSR1']) ^ 
                                      Extract(16, 16, atual['LFSR1']) ^ 
                                      Extract(13, 13, atual['LFSR1']), 
                                      Extract(18, 1, atual['LFSR1'])),
              Or(cBit1 == cBit2,cBit1 == cBit3))
    
    t1n = And(prox['LFSR1'] == atual['LFSR1'], 
              Not(Or(cBit1 == cBit2,cBit1 == cBit3)))

    t2y = And(prox['LFSR2'] == Concat(Extract(21, 21, atual['LFSR2']) ^ 
                                      Extract(20, 20, atual['LFSR2']), 
                                      Extract(21, 1, atual['LFSR2'])),
              Or(cBit1 == cBit2,cBit2 == cBit3))
    
    t2n = And(prox['LFSR2'] == atual['LFSR2'], 
              Not(Or(cBit1 == cBit2,cBit2 == cBit3)))

    t3y = And(prox['LFSR3'] == Concat(Extract(22, 22, atual['LFSR3']) ^ 
                                      Extract(21, 21, atual['LFSR3']) ^ 
                                      Extract(20, 20, atual['LFSR3']) ^ 
                                      Extract(7, 7, atual['LFSR3']), 
                                      Extract(22, 1, atual['LFSR3'])),
              Or(cBit3 == cBit2,cBit1 == cBit3))
    
    t3n = And(prox['LFSR3'] == atual['LFSR3'], 
              Not(Or(cBit3 == cBit2,cBit1 == cBit3)))

    return And(Or(t1y,t1n),Or(t2y,t2n),Or(t3y,t3n))

In [282]:
def gera_sfots(init,error,trans,k):

    solver = Solver()

    estados = [states(i) for i in range(k)]

    solver.add(init(estados[0]))

    for i in range(k):
        solver.add(Not(error(estados[i])))

    for i in range(k-1):
        solver.add(trans(estados[i],estados[i+1]))

    check = solver.check()
    
    if check == sat:
        print("Is solvable")
        m = solver.model()
        for i in range(k):
            print("Passo",i+1)
            for v in estados[i]:
                x = m.eval(estados[i][v])
                print(v,"=", format(x.as_long(),f'0{m[estados[i][v]].size()}b'))
            print("--------------------------------")
    else:
        print("Not solvable")

gera_sfots(init,error,transition,10)

Is solvable
Passo 1
LFSR1 = 0000000000011000000
LFSR2 = 0001000001000000000000
LFSR3 = 01010111000110110101010
--------------------------------
Passo 2
LFSR1 = 0000000000001100000
LFSR2 = 0000100000100000000000
LFSR3 = 01010111000110110101010
--------------------------------
Passo 3
LFSR1 = 0000000000000110000
LFSR2 = 0000010000010000000000
LFSR3 = 01010111000110110101010
--------------------------------
Passo 4
LFSR1 = 0000000000000110000
LFSR2 = 0000001000001000000000
LFSR3 = 00101011100011011010101
--------------------------------
Passo 5
LFSR1 = 0000000000000011000
LFSR2 = 0000000100000100000000
LFSR3 = 00101011100011011010101
--------------------------------
Passo 6
LFSR1 = 0000000000000001100
LFSR2 = 0000000010000010000000
LFSR3 = 00101011100011011010101
--------------------------------
Passo 7
LFSR1 = 0000000000000000110
LFSR2 = 0000000001000001000000
LFSR3 = 00101011100011011010101
--------------------------------
Passo 8
LFSR1 = 0000000000000000011
LFSR2 = 00000000001000001000

b. Use o algoritmo PDR “property directed reachability” (codifique-o ou use uma versão pré-existente) e, com ele, tente provar a segurança deste modelo.

In [283]:
def states_PDR(i):
    state = {}

    state['LFSR1'] = BitVec('LFSR1',19)
    state['LFSR2'] = BitVec('LFSR2',22)
    state['LFSR3'] = BitVec('LFSR3',23)

    return state

In [284]:
# Nesta função não descrevi o P porque o estado de erro é a negação do init, logo como o P é a negação do estado de erro, P é igual ao init
def PDR(init,trans,error):
    
    solver = Solver()

    frames = [Not(init(states_PDR(0)))]
    k = 0

    while True:
        print("Iteração",k)
        bad = get_bad_cubes(error,frames,k,solver)

        if bad == None:
            if k > 0 and frames[k] == frames[k-1]:
                print("System is Safe")
                return None
            else:
                frames.append(False)
                k+=1
        else:
            block = bloqueio(bad,frames,k,trans,solver)
            if not block:
                print("System is Unsafe")
                return None


def get_bad_cubes(error,frames,k,solver):

    estado_atual = states_PDR(k)
    solver.push()
    for frame in frames:
        solver.add(Not(frame))
    solver.add(error(estado_atual))

    if solver.check() == sat:
        model = solver.model()
        cube = {var: model.eval(var, model_completion=True) for var in estado_atual.values()}
        solver.pop()
        return cube
    
    solver.pop()
    return None

def bloqueio(bad_cube,frames,k,trans,solver):

    for i in range(k,1,-1):
        solver.push()
        estado_atual = states_PDR(i-1)
        proximo_estado = states_PDR(i)

        solver.add(trans(estado_atual,proximo_estado))
        for j in range(i):
            solver.add(Not(frames[j]))
        cube = True
        for var,val in bad_cube.items():
            cube = And(cube,var != val)

        solver.add(cube)
        
        if solver.check() == unsat:
            solver.pop()
            print("Cubo bloqueado no frame",i)
            frames[i] = cube
            return True
        solver.pop()

    return False

PDR(init,transition,error)

Iteração 0
Iteração 1
Iteração 2
System is Safe
